# AutoGen for Backdoors & Breaches

In [ ]:
import os
import json
import random
from typing import List, Dict, Union

seed = 0
random.seed(seed)

In [ ]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
llm_config = {"config_list": [{"model": "gpt-4o", "api_key": OPENAI_API_KEY}], "temperature": 1.0}

## Defining Tools¶

### Loading Cards

In [ ]:
# Load the cards from the JSON file
with open("cards.json", "r") as file:
    bnb_cards = json.load(file)["data"]

In [ ]:
bnb_cards[0]

### Drawing Incidents

In [ ]:
def draw_incident_cards() -> str:
    """
    Draws one incident card from each required type: INITIAL COMPROMISE, PIVOT and ESCALATE, C2 and EXFIL, and PERSISTENCE.
    
    :return: A formatted string listing the name, description, and detection methods of one card from each required type.
    """
    required_types = {
        "INITIAL COMPROMISE": "initial",
        "PIVOT and ESCALATE": "pivot",
        "C2 and EXFIL": "c2",
        "PERSISTENCE": "persist"
    }
    selected_cards_info = []

    for category, card_type in required_types.items():
        type_cards = [card for card in bnb_cards if card["type"] == card_type]
        
        if type_cards:
            selected_card = random.choice(type_cards)
            card_info = (
                f"Category: {category}\n"
                f"Name: {selected_card['name']}\n"
                f"Description: {selected_card['description']}\n"
                f"Detection Procedures: {', '.join(selected_card['detection'])}"
            )
            selected_cards_info.append(card_info)
    
    return "\n\n".join(selected_cards_info)

In [ ]:
incident_cards = draw_incident_cards()
print(incident_cards)

### Drawing Procedures

In [ ]:
def draw_procedure_cards() -> str:
    """
    Draws 4 random Procedure cards to establish the 'Established Procedures' and places the remaining cards as 'Other Procedures'.
    
    :return: A formatted string listing the established procedures and other procedures.
    """
    procedure_cards = [card for card in bnb_cards if card["type"] == "procedure"]
    established_procedures = random.sample(procedure_cards, 4)
    other_procedures = [card for card in procedure_cards if card not in established_procedures]
    
    established_info = ["Established Procedures (+3 modifier):"]
    for card in established_procedures:
        card_info = (
            f"Name: {card['name']}\n"
            f"Description: {card['description']}"
        )
        established_info.append(card_info)
    
    other_info = ["Other Procedures (+0 modifier):"]
    for card in other_procedures:
        card_info = (
            f"Name: {card['name']}\n"
            f"Description: {card['description']}"
        )
        other_info.append(card_info)
    
    return "\n\n".join(established_info) + "\n\n" + "\n\n".join(other_info)

In [ ]:
procedure_cards = draw_procedure_cards()
print(procedure_cards)

## Initializing Agents

### Defining the Sequence

In [ ]:
sequence_of_play = (
    "Sequence of Play:\n\n"
    
    "1. Set the Scenario:\n"
    "   - Select one card for each of the four attack stages (Initial Compromise, Pivot and Escalate, C2 and Exfil, Persistence).\n"
    "   - Craft a detailed initial scenario description based on the chosen Initial Compromise card. Provide enough context "
    "for the Defenders to understand the breach, but avoid revealing any specific details or names from the Attack cards.\n\n"
    
    "2. Introduce the Defenders to the available Procedure cards:\n"
    "   - Explain the distinction between Established Procedures (with a +3 modifier) and Other Procedures (with a +0 modifier).\n"
    "   - Inform the Defenders of the initial setup, including which procedures are classified as Established vs. Other. "
    "Note that certain procedures may shift between these categories during gameplay.\n\n"
    
    "3. Start Each Turn (Turn 1 to Turn 10):\n"
    "   - At the beginning of each turn, announce the current turn number to the Defenders.\n"
    "   - Remind Defenders of any Procedure cards on cooldown and therefore unavailable for selection. "
    "Notify them of any changes in which procedures are classified as Established vs. Other (modifier changes).\n"
    "   - Track the number of consecutive failures. If an Inject is triggered by three consecutive failures, draw an Inject card.\n"
    "   - Prompt the Defenders to discuss and select one Procedure card to use for this turn.\n\n"
    
    "4. Defenders’ Procedure Attempt:\n"
    "   - When the Defenders choose a Procedure, roll a 20-sided dice to determine if their attempt succeeds. "
    "Apply the appropriate modifier based on the type of Procedure selected:\n"
    "      - Established Procedure: +3 modifier to the roll.\n"
    "      - Other Procedure: +0 modifier to the roll.\n"
    "   - With the modifier applied, determine success or failure:\n"
    "      - Adjusted Roll 11 or higher: The attempt is successful.\n"
    "      - Adjusted Roll 10 or lower: The attempt fails.\n\n"
    
    "5. Respond to Success or Failure:\n"
    "   - On Success: Check if the Procedure used is listed under the 'Detection' methods for any of the hidden attack cards.\n"
    "      - If it matches, reveal that specific attack card to the Defenders.\n"
    "      - If multiple attack cards can be detected by the same Procedure, reveal only one and tell the Defenders they've detected "
    "a part of the breach.\n"
    "      - Reset the consecutive failure count to zero on a success.\n"
    "   - On Failure: Increase the consecutive failure count by one. Provide feedback noting that the Procedure did not reveal anything new.\n\n"
    
    "6. Triggering an Inject Event (Optional):\n"
    "   - Draw an Inject card only if any of the following specific conditions are met:\n"
    "      - A natural roll of 1 or 20 occurs (before any modifiers are applied to the dice roll), or\n"
    "      - Three consecutive procedure attempts have failed.\n"
    "   - When an Inject is triggered, draw one card from the Inject pile and reveal it to all players.\n"
    "   - Follow the instructions on the Inject card, execute its effects, and inform the Defenders of the outcomes. "
    "Ensure they understand how the Inject impacts their investigation.\n\n"
    
    "7. End Turn:\n"
    "   - Mark the Procedure card as used and enforce a cooldown period of 3 turns for that card.\n"
    "   - Track the turn count, ensuring the game does not exceed 10 turns.\n\n"
    
    "8. End Game:\n"
    "   - Victory: If Defenders reveal all four hidden attack cards within 10 turns, announce that they have successfully uncovered the breach.\n"
    "   - Loss: If the Defenders fail to reveal all attack cards by the end of the 10th turn, announce that the breach went undetected.\n"
    "   - Save a detailed game summary in JSON format, capturing all key game events and results.\n"
    "   - Type the keyword 'END_GAME' to officially conclude the game."
)

In [ ]:
print(sequence_of_play)

### Initializing the Incident Captain

In [ ]:
from autogen import ConversableAgent

In [ ]:
incident_captain = ConversableAgent(
    name="Incident_Captain",
    system_message=(
        "Welcome to Backdoors & Breaches! You are the Incident Captain, responsible for guiding the "
        "Defenders through a simulated cyber breach scenario. Your role is to control the game, craft the "
        "attack scenario, and provide guidance as Defenders attempt to detect and counter the breach.\n\n"
        
        "Your responsibilities include:\n"
        "- Selecting four hidden attack cards to define the breach scenario. These cards represent each stage: "
        "Initial Compromise, Pivot and Escalate, Command and Control, and Persistence.\n"
        "- Introducing the available Procedure cards (Established and Other) and explaining their roles and modifiers.\n"
        "- Tracking and managing game mechanics, including Procedure card cooldowns, modifier applications, and turn count.\n"
        "- Answering Defenders' questions or clarifying the scenario when asked, giving hints where appropriate.\n"
        "- Monitoring Defenders' actions and introducing injects (unexpected challenges) when triggered by certain conditions, such as critical failures.\n"
        "- Keeping the game within the 10-turn limit and ensuring Defenders have the context and support needed to achieve their objectives.\n\n"
        
        f"{sequence_of_play}\n\n"
        
        "Throughout the game, maintain the flow, stay in character, and guide Defenders with clarity. " \
        "Remind them of the cooldown status of procedures, modifier categories, and any shifts in procedure types. "
        "Do not reveal any hidden attack details unless their actions specifically uncover them. Let’s begin!\n\n"
        
        f"{'-' * 80}\n\n"
        f"The hidden attack cards for this game scenario are as follows:\n\n{incident_cards}\n\n"
        f"{'-' * 80}\n\n"
        f"The available procedure cards are divided into Established and Other Procedures:\n\n{procedure_cards}\n\n"
        f"{'-' * 80}"
    ),
    description=(
        "Oversees the Backdoors & Breaches game. Responsible for setting up the breach scenario, "
        "managing game events, and providing guidance to Defenders as they attempt to uncover hidden attack stages. "
        "Ensures gameplay runs smoothly and that Defenders stay engaged and on task."
    ),
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "END_GAME" in msg["content"],
)

In [ ]:
print(incident_captain.system_message)

In [ ]:
print(incident_captain.description)

### Loading Defender Roles

In [ ]:
# Load the roles from the JSON file
with open("roles.json", "r") as file:
    defender_roles = json.load(file)

In [ ]:
defender_roles.keys()

### Generating System Messages

In [ ]:
def generate_system_message(role_name, role_data):
    """
    Generate the system message for a defender based on their role.
    """
    role_responsibilities = '\n'.join(['- ' + resp for resp in role_data['responsibilities']])
    return (
        f"Welcome to Backdoors & Breaches! You are a {role_name}. In this game, Defenders collaborate to uncover hidden stages "
        "of a simulated cyber attack. Your goal, along with the other Defenders, is to work together to identify and reveal four hidden attack "
        "cards within 10 turns to win the game. Each attack card represents a critical stage in the breach process that attackers might use against your organization.\n\n"
        
        "Game Overview:\n"
        "The game begins with the Incident Captain setting up the scenario by selecting four hidden attack cards representing the stages of a breach: "
        "Initial Compromise, Pivot and Escalate, Command and Control (C2), and Persistence. Defenders take turns selecting Procedure cards to investigate "
        "and uncover these stages. Procedure cards are divided into Established cards, which provide a +3 modifier to dice rolls, and Other cards, which do not provide modifiers. "
        "Each turn, the team selects one Procedure card, rolls a 20-sided dice, and applies any modifiers to determine success or failure.\n\n"
        
        "Game Mechanics:\n"
        "- Procedure Cards: Represent investigative approaches. Established cards have a +3 modifier, while Other cards have no modifier.\n"
        "- Dice Rolling: After selecting a Procedure, roll a 20-sided dice and apply the modifier. "
        "A final roll of 11 or higher results in success, while 10 or lower results in failure.\n"
        "- Outcomes: Success reveals a hidden attack card if the Procedure matches its detection methods. "
        "Failures contribute to a consecutive failure count, which may trigger Inject events, introducing unexpected challenges.\n"
        "- Cooldown Period: After a Procedure card is used, regardless of the outcome, it enters a 3-turn cooldown period during which it cannot be selected again.\n\n"
        
        f"Your Responsibilities as a {role_name}:\n"
        "- Collaborate with your teammates to analyze the scenario and decide the most effective Procedures to use each turn.\n"
        "- Provide your insights, expertise, or support based on your specific role and knowledge level.\n"
        "- Stay engaged, communicate effectively, and contribute to the success of your team.\n"
        "- Adapt to new information, including the outcomes of Procedure attempts and any Inject events introduced during the game.\n"
        f"{role_responsibilities}\n\n"
        
        "Victory Condition:\n"
        "The Defenders win by successfully uncovering all four attack cards within 10 turns. If the Defenders fail to do so, the breach remains undetected, "
        "and the game is lost.\n\n"
        
        "Your role is crucial to the team's success. Work together, strategize effectively, and let's uncover the breach!"
    )

In [ ]:
print(generate_system_message(role_name='Team Leader', role_data=defender_roles['Team Leader']))

### Initializing the Defenders

In [ ]:
team_structures = {
    "Homogeneous Centralized": {
        "Team Leader": 1,
        "Team Member": 4,
    },
    "Heterogeneous Centralized": {
        "Team Leader": 1,
        "Endpoint Security Expert": 1,
        "Network Traffic Analysis Expert": 1,
        "Log and Behavioral Analysis Expert": 1,
        "Deception and Containment Expert": 1,
    },
    "Homogeneous Decentralized": {
        "Team Member": 5,
    },
    "Heterogeneous Decentralized": {
        "Endpoint Security Expert": 1,
        "Network Traffic Analysis Expert": 1,
        "Log and Behavioral Analysis Expert": 1,
        "Deception and Containment Expert": 1,
        "Incident Response Expert": 1
    },
    "Homogeneous Hybrid": {
        "Expert": 3,
        "Beginner": 2,
    },
    "Heterogeneous Hybrid": {
        "Endpoint Security Expert": 1,
        "Network Traffic Analysis Expert": 1,
        "Log and Behavioral Analysis Expert": 1,
        "Beginner": 2,
    }
}

In [ ]:
def create_defender_agents(team_structure_name, defender_roles, team_structures):
    """
    Create defender agents based on the specified team structure.
    """
    if team_structure_name not in team_structures:
        raise ValueError(f"Unknown team structure: {team_structure_name}")

    structure = team_structures[team_structure_name]
    defenders = []

    for role_name, count in structure.items():
        for i in range(count):
            if count == 1:
                agent_name = f"{role_name} Defender"
            else:
                agent_name = f"{role_name} Defender {i + 1}"
            system_message = generate_system_message(agent_name, defender_roles[role_name])
            defender = ConversableAgent(
                name=agent_name.replace(' ', '_'),
                system_message=system_message,
                description=defender_roles[role_name]["description"],
                llm_config=llm_config,
                human_input_mode="NEVER"
            )
            defenders.append(defender)

    return defenders

In [ ]:
team_structure_name = "Homogeneous Centralized"
defenders = create_defender_agents(
    team_structure_name=team_structure_name,
    defender_roles=defender_roles,
    team_structures=team_structures,
)

In [ ]:
for defender in defenders:
    print('=' * 80)
    print(defender.name)
    print('-' * 80)
    print(defender.system_message)
    print('-' * 80)
    print(defender.description)
    print('=' * 80)

## Defining More Tools

### Drawing an Injection

In [ ]:
def draw_inject_card() -> str:
    """
    Draws a random Injection Card from the available cards.
    
    :return: A formatted string listing the name, description, and any additional details of the drawn inject card.
    """
    inject_cards = [card for card in bnb_cards if card["type"] == "inject"]
    selected_card = random.choice(inject_cards)
    
    inject_card_info = (
        f"Inject Card:\n"
        f"Name: {selected_card['name']}\n"
        f"Description: {selected_card['description']}"
    )
    
    return inject_card_info

In [ ]:
print(draw_inject_card())

### Rolling a Dice

In [ ]:
def roll_dice() -> int:
    """
    Rolls a 20-sided dice and returns the result.

    :return: A random integer between 1 and 20, inclusive, representing the dice roll outcome.
    """
    return random.randint(1, 20)

In [ ]:
roll_dice()

### Saving Results

In [ ]:
base_filename = f"{team_structure_name}_{seed}".replace(' ', '_').lower()
print(base_filename)

In [ ]:
def save_game_results(
    incident_cards: List[str],
    procedure_cards: Dict[str, List[str]],
    turn_results: List[Dict[str, Union[int, str, bool]]],
    final_result: str,
    defenders: List[str],
) -> str:
    """
    Save the game results to a JSON file in the 'results' folder, with structured order.

    :param incident_cards: List of incident card names, sorted by attack stage (strings only).
    :param procedure_cards: Dictionary with keys "established" and "other", each containing a list of procedure card names (strings only).
    :param turn_results: List of dictionaries for each turn, including: "turn" (int), "procedure" (str), "dice_roll" (int), 
        "modifier" (int), "success" (bool), "consecutive_failures" (int), optional "revealed_incident" (str) if an incident 
        card was revealed, and optional "inject" (str) if triggered.
    :param final_result: String indicating the final game outcome, either "Victory" or "Loss".
    :param defenders: List of defender agent names (strings only) who participated in the game.
    :return: A confirmation message with the path of the saved game results file.
    """

    # Ensure the folder exists
    os.makedirs('results', exist_ok=True)

    # Calculate consecutive failures for each turn
    failure_streak = 0
    for turn in turn_results:
        if not turn.get("success", False):
            failure_streak += 1
        else:
            failure_streak = 0
        turn["consecutive_failures"] = failure_streak

    # Prepare game data in structured order
    game_data = {
        "defenders": defenders,
        "incident_cards": incident_cards,
        "procedure_cards": {
            "established": procedure_cards.get("established", []),
            "other": procedure_cards.get("other", [])
        },
        "turn_results": turn_results,
        "summary": {
            "total_turns_played": len(turn_results),
            "final_result": final_result
        }
    }

    # Save data to JSON file
    filename = f"results/results_{base_filename}.json"
    with open(filename, 'w') as f:
        json.dump(game_data, f, indent=2)

    # Return confirmation message
    return f"Game results saved to {filename}"

## Registering Tools

In [ ]:
from autogen import UserProxyAgent, register_function

In [ ]:
tool_executor = UserProxyAgent(
    name="Tool_Executor",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER",
    description=(
        "A dedicated agent responsible for executing specific game functions. "
        "Handles tool-related requests from the Incident Captain, such as drawing cards and rolling a dice "
        "as needed during gameplay. This agent operates silently, "
        "only responding to tool execution calls without participating in general discussions."
    )
)

In [ ]:
print(tool_executor.description)

In [ ]:
for tools in [draw_inject_card, roll_dice, save_game_results]:
    register_function(
        f=tools,
        caller=incident_captain,
        executor=tool_executor,
        name=tools.__name__,
        description=tools.__doc__,
    )

In [ ]:
incident_captain.llm_config["tools"]

## Creating a Group Chat

In [ ]:
from autogen import GroupChat, GroupChatManager

In [ ]:
allowed_transitions = {
    incident_captain: [incident_captain, tool_executor] + defenders,
    tool_executor: [incident_captain],
}
allowed_transitions.update({
    defender: [incident_captain] + defenders for defender in defenders
})

In [ ]:
group_chat = GroupChat(
    agents=[incident_captain, tool_executor] + defenders,
    messages=[],
    max_round=1000,
    send_introductions=True,
    speaker_selection_method="auto",
    speaker_transitions_type="allowed",
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
)

In [ ]:
group_chat_manager = GroupChatManager(
    name="Group_Chat_Manager",
    groupchat=group_chat,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "END_GAME" in msg["content"],
)

## Playing the Game

In [ ]:
chat_result = group_chat_manager.initiate_chat(
    recipient=incident_captain,
    message=(
        "Welcome to the Backdoors & Breaches game! You are the Incident Captain. "
        "Please begin by setting the stage for our Defenders: select and describe the breach scenario based on the "
        "Initial Compromise card, providing context without revealing specific details.\n\n"
        "Once the scenario is set, introduce the available Procedure cards, including which are Established and "
        "which are Other Procedures. The Defenders are ready and waiting to start their investigation."
    ),
    max_turns=None,
)

## Saving Messages

In [ ]:
os.makedirs('results', exist_ok=True)
chat_filename = f"results/chat_{base_filename}.json"
print(chat_filename)

In [ ]:
with open(chat_filename, 'w') as f:
    json.dump(incident_captain.chat_messages[group_chat_manager], f, indent=2)
print(f"Chat messages saved to {chat_filename}")